In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D


In [ ]:
# Step 2: Download stock data
stock_data = yf.download('AAPL', start='2020-01-01', end='2024-01-01')
prices = stock_data['Close'].values.reshape(-1, 1)

In [ ]:
# Step 3: Preprocessing - Scaling data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(scaled_prices, seq_length)


In [ ]:
# Step 4: Build Transformer Model in TensorFlow
def build_transformer_model(seq_length, input_dim, embed_dim=64, num_heads=4, ff_dim=128, dropout_rate=0.1):
    inputs = Input(shape=(seq_length, input_dim))

    # Linear projection (embedding)
    x = Dense(embed_dim)(inputs)

    # Multi-Head Attention Layer
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(x + attn_output)

    # Feed Forward Network
    ff_output = Dense(ff_dim, activation='relu')(out1)
    ff_output = Dense(embed_dim)(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ff_output)

    # Global Pooling & Output
    pooled = GlobalAveragePooling1D()(out2)
    outputs = Dense(1)(pooled)

    model = tf.keras.Model(inputs, outputs)
    return model

model = build_transformer_model(seq_length, input_dim=1)
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# Step 5: Train the model
history = model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

In [ ]:
# Step 6: Predict & Visualize
predicted = model.predict(X)
predicted_prices = scaler.inverse_transform(predicted)
actual_prices = scaler.inverse_transform(y)

plt.figure(figsize=(12, 6))
plt.plot(actual_prices, label='Actual Price', color='blue')
plt.plot(predicted_prices, label='Predicted Price', color='orange')
plt.legend()
plt.title('Stock Price Prediction using Transformer (TensorFlow)')
plt.show()

In [15]:
# Apple stock (AAPL)
aapl = yf.Ticker("AAPL")
# Fetch past 5 days, 30-minute interval data
df = aapl.history(period="60d", interval="30m")
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-12-03 09:30:00-05:00,239.919998,240.919998,238.900299,240.889999,5378912,0.0,0.0
2024-12-03 10:00:00-05:00,240.860001,241.369904,240.360001,240.470001,3169396,0.0,0.0
2024-12-03 10:30:00-05:00,240.475006,241.240005,240.449997,241.190506,2412482,0.0,0.0
2024-12-03 11:00:00-05:00,241.190002,241.339996,240.809998,241.330002,1549854,0.0,0.0
2024-12-03 11:30:00-05:00,241.330002,241.537994,240.981995,241.410004,1841667,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-03 13:30:00-05:00,241.020004,241.429993,240.440002,241.320007,2197239,0.0,0.0
2025-03-03 14:00:00-05:00,241.309998,241.869995,240.910904,241.151794,1518981,0.0,0.0
2025-03-03 14:30:00-05:00,241.160004,242.000000,239.880005,240.289993,1983194,0.0,0.0
